In [124]:
# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# uncomment once you paste your mypytable.py into mysklearn package
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

## Load and clean the data

In [125]:
mytable = MyPyTable()
fName = os.path.join("input_data", "trimmed_data.csv")
dataset = mytable.load_from_file(fName)
dataset.remove_rows_with_missing_values()
rows_to_delete = ['Review Count', 'Attributes', 'Review Length']
myutils.remove_rows_from_data(rows_to_delete, dataset)
print(dataset.column_names)
myutils.get_friend_count(2, dataset)

['Fans', 'Compliment Plain', 'Friends', 'Useful']


## Stratified k-fold Accuracy Check

In [126]:
#==================================
#      Get stratified training    =
#      and testing sets           =
#==================================
k = 10
X, y = myutils.split_x_y_train(dataset.data)
x_train, x_test, y_train, y_test = myevaluation.train_test_split(X, y, shuffle=True)

# Get the training and testing folds
train_folds, test_folds = myevaluation.stratified_kfold_cross_validation(X, y, k)

# Get the traininga and testing sets from the folds
x_train, y_train, x_test, y_test = myutils.get_values_from_folds(X, y, train_folds, test_folds)


#==================================
#      Naive Bayes Classifier     =
#==================================
myNB = MyNaiveBayesClassifier()
myNB.fit(x_train, y_train)

# Compare predicted with actual
y_predict = myNB.predict(x_test)
count = 0
for i in range(len(y_predict)):
    binned_predict = myutils.get_useful_bin(y_predict[i])
    binned_test = myutils.get_useful_bin(y_test[i])
    if (binned_predict == binned_test):
        count = count + 1;

# Calculate accuracy and error
accuracy = count / len(y_predict)
error = (len(y_predict) - count) / len(y_predict)

print("Naive Bayes: accuracy =", accuracy, "error =", error)

myDT = MyDecisionTreeClassifier()
myDT.fit(x_train, y_train)

y_predict = myDT.predict(x_test)
count = 0
for i in range(len(y_predict)):
    binned_predict = myutils.get_useful_bin(y_predict[i])
    binned_test = myutils.get_useful_bin(y_test[i])
    if (binned_predict == binned_test):
        count = count + 1;

accuracy = count / len(y_predict)
error = (len(y_predict) - count) / len(y_predict)

print("Decision Tree: accuracy =", accuracy, "error =", error)

Naive Bayes: accuracy = 0.8598130841121495 error = 0.14018691588785046
Decision Tree: accuracy = 0.16822429906542055 error = 0.8317757009345794


## Confusion Matrices

In [127]:
# Get the x values to perform the matrix over
x = []
for i in range(10):
    x.append(i + 1)

actual = []
predicted = []
for i in range(len(y_test)):
    actual.append(myutils.get_useful_bin(y_test[i]))
    predicted.append(myutils.get_useful_bin(y_predict[i]))

# Get the matrix from stratified
matrix = myevaluation.confusion_matrix(actual, predicted, x)

# Make the table header and calculate the statistics
table_header = ['Useful', 1,2,3,4,5,6,7,8,9,10, 'Total', 'Recognition (%)']
complete_matrix = myutils.calc_matrix_stats(matrix, False)

print()
print('Naive Bayes')
myutils.print_tabulate(complete_matrix, table_header)


Naive Bayes
========  ===  ===  ===  ===  ===  ===  ===  ===  ===  ====  =======  =================
  Useful    1    2    3    4    5    6    7    8    9    10    Total    Recognition (%)
========  ===  ===  ===  ===  ===  ===  ===  ===  ===  ====  =======  =================
       1  170   17    4    3    4    3    0    1    0     0      202              84.16
       2  157   41    2    4    1    7    2    1    0     0      215              19.07
       3  155   10    6    2    0    0    0    0    0     0      173               3.47
       4  125    6    3    4    0    0    0    0    0     0      138               2.9
       5  165   63   15   13    1    1    0    0    0     0      258               0.39
       6   41   55   37   30   13    7    0    0    0     0      183               3.83
       7   24   17    6   14   16    4    0    0    0     0       81               0
       8   15   10   10   13   11    5    0    0    0     0       64               0
       9   16    5    2   